In [5]:
import matplotlib.pyplot as plt
import pathlib
import seaborn as sns
import wfdb

from ecg_hrv_analysis.frequency_domain import *
from ecg_hrv_analysis.plot import *
from ecg_hrv_analysis.signal_processing import *
from ecg_hrv_analysis.time_domain import *
from scipy.stats import norm



SyntaxError: invalid syntax (<ipython-input-5-5078d172d2e0>, line 5)

In [ ]:

data_path = pathlib.Path("./ecg_data/physionet.org/files/mitdb/1.0.0")

patiens, annotations = data_dict(data_path)

rpeaks_dict, fpfn_R_an_dict, fpfn_R_R_dict = rpeaks_extr(pat, an, low, high)

In [ ]:
i_pat = 0
chan = 0
f_low = 5
f_high = 12

ecg_signals(i_pat, patients ,chan, f_low, f_high )

In [ ]:
length = len( [i for i, elem in enumerate(r_peaks) if elem<5400])

xx = [rp/360 for rp in r_peaks[:length]]
xx2 = [rp/360 for rp in an[0].get('sample')[1:length +1 ]]

yy= mwi_ecg[r_peaks[:length]] 
yy2= mwi_ecg[ an[0].get('sample')[1:length+1]]

fig, axs = plt.subplots(1, 1, figsize= (12,7))
axs.plot(x[:5400],mwi_ecg[:5400])
axs.scatter( xx, yy, c="r", marker='x',label = 'detected R peaks' )
axs.scatter( xx2, yy2, c="g", marker='x',label = 'Annotations' )
plt.legend()
plt.xlabel('time [s]')
plt.ylabel('MWI ECG')

In [ ]:
hr1 = calculate_heart_rate(r_peaks, 360)

an_peaks = an[0].get('sample')
hr2 = calculate_heart_rate(an_peaks,360)


In [ ]:
fig, axs = plt.subplots(3, 1, sharex = True ,figsize = (12,7), gridspec_kw={'height_ratios': [7, 1,1]})

min_val = min(min(hr1), min(hr2))-5
max_val = max(max(hr1), max(hr2[1:]))+5
# bins = np.linspace(min_val, max_val, num=150)

mu1, std1 = norm.fit(hr1)
mu2, std2 = norm.fit(hr2)

axs[0].hist(hr1, bins=100, histtype=u'step', color = 'r')
axs[0].hist(hr1, bins=100, alpha=0.3, label = 'Heart rate from detected R, \n mean HR %.2f' %mu1, color = 'r')
axs[0].hist(hr2[1:], bins=100, histtype=u'step', color = 'blue')
axs[0].hist(hr2[1:], bins=100, alpha=0.3, label = 'Heart rate from annotated R, \n mean HR %.2f' %mu2, color = 'blue')
sns.boxplot(hr1, ax = axs[1],color = 'r', boxprops=dict(alpha=.3), orient="h")
sns.boxplot(hr2[1:], ax = axs[2],color = 'blue', boxprops=dict(alpha=.3), orient="h")

axs[1].axis('off')
axs[2].axis('off')

axs[0].tick_params(reset=True)
plt.xlim(min_val, max_val)
axs[0].legend()

In [ ]:
test_dict = count_beats(pat, an, num = True)

# create sample data with labels
data = test_dict
df = pd.DataFrame(data, index=[i for i in range(48)])

# create stacked bar plot
ax = df.plot(kind='bar', stacked=True, figsize=(8,6), cmap = 'PiYG')

# add labels and legend
ax.set_xlabel('Individuals')
ax.set_ylabel('Number of Labels')
ax.legend(title='Labels')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# show plot
plt.show()

In [ ]:
hr_m = mean_heart_rate(rpeaks_dict, 360)
means = mean_for_an()

fig, axs = plt.subplots(1, 1, figsize = (15,10))
axs.plot([i for i in range(len(pat))],hr_m, color = 'b')
axs.plot([i for i in range(len(pat))],means, color = 'r')
axs.scatter([i for i in range(len(pat))],hr_m,color = 'b',label = 'Mean heart rate')
axs.scatter([i for i in range(len(pat))],means,color = 'r',  label = 'Mean heart rate from an.')
plt.legend()
plt.xlabel('Individuals')
plt.ylabel('Mean heart rate')

In [ ]:
test_dict = count_beats(pat, an, num = False)

df = pd.DataFrame.from_dict(test_dict)
fig, ax = plt.subplots(1,1,figsize = (16,9))
ax = sns.heatmap(df,linewidth=0., cmap ="PiYG")